In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as spo

import lyaemu.likelihood as lyl

%matplotlib inline


/share/apps/anaconda/python3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/keir/Software/lya_emulator/lyaemu/SimulationRunner/SimulationRunner/cambpower.py:8: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/share/apps/anaconda/python3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/share/apps/anaconda/python3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/share/apps/an

In [2]:
#Load linear power spectra
fname = '/Users/kwame/Software/axionCAMB/axion_%s_matterpower_z_99.dat'
mass_22 = np.array([0.5, 1., 2., 4., 5., 10., 20., 40., 50., 100., 200., 400., 500., 1000., 2000., 4000., 5000.]) #, 10000.]) #, 20000., 50000., 100000.])

linear_power = [None] * (mass_22.size + 1)
linear_power[0] = np.loadtxt(fname%'CDM')

transfer_function = np.zeros((mass_22.size, linear_power[0].shape[0]))
nCDM_parameters = np.zeros((mass_22.size, 3))
nCDM_covariance = np.zeros((mass_22.size, 3, 3))

plt.figure(figsize=(12., 8.))

for i, mass in enumerate(mass_22):
    linear_power[i+1] = np.loadtxt(fname%str(int(mass)))
    transfer_function[i, :] = np.sqrt(linear_power[i+1][:, 1] / linear_power[0][:, 1])
    #Fit nCDM parameters
    nCDM_parameters[i, :], nCDM_covariance[i, :, :] = spo.curve_fit(lyl.transfer_function_nCDM,
                                                        linear_power[i+1][:, 0], transfer_function[i, :],
                                                        p0=np.array([0.05, 5., -5.]))
    print(mass, nCDM_parameters[i, :], np.sqrt(np.diag(nCDM_covariance[i, :, :])) / nCDM_parameters[i, :])

    plt.plot(linear_power[i+1][:, 0], transfer_function[i, :], label=r'%f e-22 eV'%mass)
    plt.plot(linear_power[i+1][:, 0], lyl.transfer_function_nCDM(linear_power[i+1][:, 0], nCDM_parameters[i, 0],
                                        nCDM_parameters[i, 1], nCDM_parameters[i, 2]), ls='--')

plt.xscale('log')
plt.xlim([1.e-1, 1.e+4])
plt.xlabel(r'k (h / Mpc)')
plt.ylabel(r'T(k)')
plt.axhline(y=0., ls=':', color='black')
plt.axhline(y=1., ls=':', color='black')
plt.legend()


OSError: /Users/kwame/Software/axionCAMB/axion_CDM_matterpower_z_99.dat not found.

In [ ]:
#Fit nCDM_parameters = f(log_mass)
def alpha_model(log_mass, b, a, m, c):
    """Model for alpha as a function of log mass"""
    return 10. ** ((b * (log_mass ** 3)) + (a * (log_mass ** 2)) + (m * log_mass) + c)

def beta_model(log_mass, a, m, c):
    """Model for beta as a function of log mass"""
    return (a * (log_mass ** 2)) + (m * log_mass) + c

def gamma_model(log_mass, b, a, m, c):
    """Model for gamma as a function of log mass"""
    return -1. * (10. ** ((b * (log_mass ** 3)) + (a * (log_mass ** 2)) + (m * log_mass) + c))


In [ ]:
#Study nCDM parameters
figure, axes = plt.subplots(nrows=3, ncols=1, figsize=(12., 12.), sharex=True)
y_label = [r'log10(alpha [Mpc / h])', r'beta', r'log10(-1 * gamma)']

#Fit nCDM model
models = [alpha_model, beta_model, gamma_model]
p0 = [np.array([0., 0., -1., -22.]), np.array([0., -1., -22.]), np.array([0., 0., -1., -22.])]
log_mass_plot = np.linspace(-22.5, -18., num=1000)

for i in range(3):
    model_parameters, model_covariance = spo.curve_fit(models[i], np.log10(mass_22 * 1.e-22), nCDM_parameters[:, i], 
                                                          p0=p0[i], maxfev=8000)
    print(y_label[i], model_parameters, np.sqrt(np.diag(model_covariance)) / model_parameters)
    y_model = models[i](log_mass_plot, *model_parameters)
    
    y_data = nCDM_parameters[:, i]
    if (i == 0):
        y_data = np.log10(y_data)
        y_model = np.log10(y_model)
    if (i == 2):
        y_data = np.log10(-1. * y_data)
        y_model = np.log10(-1. * y_model)
    
    axes[i].scatter(np.log10(mass_22 * 1.e-22), y_data)
    if i == 0:
        hubble_planck = 0.6686
        hubble = 0.74279
        axes[i].scatter(np.log10(mass_22 * 1.e-22), y_data + np.log10(hubble / hubble_planck),
                        label=r'h = %f'%hubble)
        hubble = 0.65932
        axes[i].scatter(np.log10(mass_22 * 1.e-22), y_data + np.log10(hubble / hubble_planck),
                        label=r'h = %f'%hubble)
    axes[i].plot(log_mass_plot, y_model, ls='--')
    axes[i].set_ylabel(y_label[i])

axes[0].axhline(y=-1., ls=':', color='black')
#axes[0].axhline(y=0., ls=':', color='black')
axes[2].axhline(y=0., ls=':', color='black')
#axes[2].axhline(y=-10., ls=':', color='black')

axes[0].legend()

#axes[2].set_ylim([-11., 1.])
axes[2].set_xlabel(r'log10(mass [eV])')
